****Introduction****
Tomorrow there will have a final match between England and Italy in Euro 2021, below is a simple model built to predict a team can be a winner in last match.
**Note: a model's result should be considered as reference, study and share knowledges to everyone not use it for betting or something else that can break the law**
ok! Just funny and relax a final match 

In [ ]:
#Basic Libraries
import numpy as np
import pandas as pd

# Visualizations Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.figure_factory as ff

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
all_matches = pd.read_csv(r'/kaggle/input/uefa-euro-cup-19602016/All Matches.csv')
all_matches.head()

- Date: Date on which the match was played
- HomeTeamName: Name of the Home Team
- AwayTeamName:	Name of the Away team
- HomeTeamGoals: Goals scored by Home Teams	
- AwayTeamGoals: Goals scored by Away Teams
- Stage: The stage of the competition when the match was played.
- SpecialWinConditions:	Special winning conditions such as away goals, penalties, etc.
- Stadium: Name of the stadium
- City:	Name of the city
- Attendance: Total number of audience in the stadium
- Year: Year on which the match was played

In [ ]:
national_tapp = pd.read_csv(r'/kaggle/input/uefa-euro-cup-19602016/National Teams Appearance.csv') 
national_tapp.head()

- Team:	Name of the teams
- Appearances: Number of times a team qualified and played in the competition
- Record streak: Highest number of times, qualifying and playing in the competition	
- Active streak: Current streak of the number of appearance	
- Debut: Debut year of a national team in the competition	
- Most recent: Most recent participation year
- Best result: Best result of a national team in the competition

In [ ]:
general_stat = pd.read_csv(r'/kaggle/input/uefa-euro-cup-19602016/Participated Teams General Statistics.csv')
general_stat.head()

- Team:	Names of the teams
- Participations: Number of times a team participated in the competition	
- Played: Number of matches played
- Win: Number of times the team won
- Draw:	Number of times, the match was a draw
- Loss:	Number of times the team lost
- Goal_For: Number of goals scored by the team
- Goal_Against:	Number of goals conceded by the team
- Goal_Difference:	Goal_For - Goal_Against
- Points: Total points the team earned from the matchesGoal_Against
- Points/match: Points earned per match

In [ ]:
top_goalscr = pd.read_csv(r'/kaggle/input/uefa-euro-cup-19602016/Top Goal Scorers.csv')
top_goalscr.head()

- Player: Name of the player	
- Team: Name of the team the player represents	
- Goals scored:	Number of goals scored
- Matches played: Number of matches played
- Goals average: Number of goals per matches	
- Tournament(s): Year of tournaments played by the player

# All Matches Exploratory

In [ ]:
# country
list_country = pd.unique(all_matches['HomeTeamName'])
for i in range(len(list_country)):
    list_country[i] = list_country[i].strip('\xa0')


In [ ]:
all_matches["Result"] = all_matches['HomeTeamGoals'] - all_matches['AwayTeamGoals']
all_matches['Result'] = all_matches['Result'].astype(str)

In [ ]:
for i in range(len(all_matches['Result'])): 
    if '-' in all_matches['Result'][i]:
      all_matches['Result'][i] = "A"
    elif all_matches['Result'][i] == '0':
        all_matches['Result'][i] = 'D'
    else:
        all_matches['Result'][i] = 'H'

In [ ]:
all_matches.head()

In [ ]:
result_per_y = all_matches.groupby(['Year'])['Result'].value_counts()

In [ ]:

trace1 = go.Bar(
    x=result_per_y.loc[:,'H'].index,
    y=result_per_y.loc[:,'H'].values,
    name='HomeWin',
    #orientation = 'h',
    marker=dict(
        color='pink'
    )
)
trace2 = go.Bar(
    x=result_per_y.loc[:,'D'].index,
    y=result_per_y.loc[:,'D'].values,
    name='Draw',
    #orientation = 'h',
    marker=dict(
        color='darkblue'
    )
)
trace3 = go.Bar(
    x=result_per_y.loc[:,'A'].index,
    y=result_per_y.loc[:,'A'].values,
    name='AwayWin',
    #orientation = 'h',
    marker=dict(
        color='red'
    )
)
data = [trace1, trace2, trace3]
layout = go.Layout(
    title='General Statistic About Results In Each Euro Season',
    font=dict(
        size=16
    ),
    barmode = 'stack',
    legend=dict(
        font=dict(
            size=16
        )
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
d= pd.DataFrame(all_matches['Result'].value_counts())
fig = px.pie(d,values='Result',names=['HomeTeamWin','Draw','AwayTeamWin'],hole=0.4,opacity=0.6,
            color_discrete_sequence=['#17869E','#4F7942','#FFBF00'],
             labels={'label':'Result','Result':'No. Of Samples'})

fig.add_annotation(text='Result',
                   x=0.5,y=0.5,showarrow=False,font_size=14,opacity=0.7,font_family='monospace')

fig.update_layout(
    font_family='monospace',
    title=dict(text='The Percentage Of Each Result 1960-2016',x=0.47,y=0.98,
               font=dict(color='#636363',size=20)),
    legend=dict(x=0.37,y=-0.05,orientation='h',traceorder='reversed'),
    hoverlabel=dict(bgcolor='white'))

fig.update_traces(textposition='outside', textinfo='percent+label')

iplot(fig)

In [ ]:
stage_result = all_matches[['Stage','Result']]
three_stage = stage_result[(stage_result['Stage'] == 'Semi-finals') | (stage_result['Stage'] == 'Final') | (stage_result['Stage'] == 'Quarter-finals')]
three_stage_c = three_stage.groupby(['Stage'])['Result'].value_counts()

In [ ]:
three_stage_c

In [ ]:
# visualize three stage result
trace1 = go.Bar(
    x=three_stage_c.loc[:,'H'].index,
    y=three_stage_c.loc[:,'H'].values,
    name='HomeWin',
    #orientation = 'h',
    marker=dict(
        color='pink'
    )
)
trace2 = go.Bar(
    x=three_stage_c.loc[:,'D'].index,
    y=three_stage_c.loc[:,'D'].values,
    name='Draw',
    #orientation = 'h',
    marker=dict(
        color='darkblue'
    )
)
trace3 = go.Bar(
    x=three_stage_c.loc[:,'A'].index,
    y=three_stage_c.loc[:,'A'].values,
    name='AwayWin',
    #orientation = 'h',
    marker=dict(
        color='red'
    )
)
data = [trace1, trace2, trace3]
layout = go.Layout(
    title='Result In Three Stage: Final, Semi Finals And Quarter Finals In 90 Minutes',
    font=dict(
        size=13
    ),
    legend=dict(
        font=dict(
            size=16
        )
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
total_result = all_matches[['HomeTeamName','AwayTeamName','Stage','SpecialWinConditions','Year','Result']]
total_result = total_result[~total_result['SpecialWinConditions'].isnull()]

In [ ]:
# split team win out of 'SpecialWinConditions' column
for i in range(len(total_result)):
    total_result['SpecialWinConditions'].iloc[i] = total_result['SpecialWinConditions'].iloc[i].replace('\xa0',"")
    lss = total_result['SpecialWinConditions'].iloc[i].split(" win", maxsplit=1)
    total_result['SpecialWinConditions'].iloc[i] = lss[0]

In [ ]:
# convert result in 90 minutes to result in full match (speacial conditions such as penatiles, extra time)
for i in range(len(total_result)):
    # clean string before compare
    if total_result['SpecialWinConditions'].iloc[i].strip() == total_result['HomeTeamName'].iloc[i].replace('\xa0',""):
        total_result['Result'].iloc[i] = 'H'
    else:
        total_result['Result'].iloc[i] = 'A'

In [ ]:
non_cond_result = all_matches[['HomeTeamName','AwayTeamName','Stage','SpecialWinConditions','Year','Result']]
non_cond_result = non_cond_result[non_cond_result['SpecialWinConditions'].isnull()]
non_cond_result = non_cond_result[(non_cond_result['Stage'] == 'Semi-finals') | (non_cond_result['Stage'] == 'Final') | (non_cond_result['Stage'] == 'Quarter-finals')]

In [ ]:
total_result = total_result[(total_result['Stage'] == 'Semi-finals') | (total_result['Stage'] == 'Final') | (total_result['Stage'] == 'Quarter-finals')]
final_result = pd.concat([non_cond_result,total_result])
final_result

In [ ]:
full_match_stage = final_result.groupby(['Stage'])['Result'].value_counts()

In [ ]:
# visualize three stage result
trace1 = go.Bar(
    x=full_match_stage.loc[:,'H'].index,
    y=full_match_stage.loc[:,'H'].values,
    name='HomeWin',
    #orientation = 'h',
    marker=dict(
        color='pink'
    )
)
trace2 = go.Bar(
    x=full_match_stage.loc[:,'A'].index,
    y=full_match_stage.loc[:,'A'].values,
    name='AwayWin',
    #orientation = 'h',
    marker=dict(
        color='darkblue'
    )
)
data = [trace1, trace2]
layout = go.Layout(
    title='Result In Three Stage: Final, Semi Finals And Quarter Finals In Full Match',
    font=dict(
        size=13
    ),
    legend=dict(
        font=dict(
            size=16
        )
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [ ]:
fullm_per_y = final_result.groupby(['Year'])['Result'].value_counts()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=fullm_per_y.loc[:,'H'].index, y=fullm_per_y.loc[:,'H'].values,
                    mode='lines+markers',
                    name='HomeWin'))
fig.add_trace(go.Scatter(x=fullm_per_y.loc[:,'A'].index, y=fullm_per_y.loc[:,'A'].values,
                    mode='lines+markers',
                    name='AwayWin'))
fig.update_layout(title='The Change In Full Match Of Three Stage Result Since 1960',
                   xaxis_title='Year',
                   yaxis_title='The Number Of Win')
iplot(fig)

In [ ]:
detail_each_stage = final_result.groupby(['Year','Stage'])['Result'].value_counts()
#final
final_stage_rp = pd.merge(detail_each_stage.loc[slice(1960,2016),['Final'],['H']],detail_each_stage.loc[slice(1960,2016),['Final'],['A']]
,how='outer', on=["Year", "Stage"]).sort_values(by=['Year'],ascending=True)
final_stage_rp.fillna(0, inplace=True)
final_stage_rp.rename(columns={'Result_x':'HomeWin','Result_y':'AwayWin'},inplace=True)

#semi_final
semi_rp = pd.merge(detail_each_stage.loc[slice(1960,2016),['Semi-finals'],['H']],detail_each_stage.loc[slice(1960,2016),['Semi-finals'],['A']]
,how='outer', on=["Year", "Stage"]).sort_values(by=['Year'],ascending=True)
semi_rp.fillna(0, inplace=True)
semi_rp.rename(columns={'Result_x':'HomeWin','Result_y':'AwayWin'},inplace=True)

#quarter
quarter_rp = pd.merge(detail_each_stage.loc[slice(1960,2016),['Quarter-finals'],['H']],detail_each_stage.loc[slice(1960,2016),['Quarter-finals'],['A']]
,how='outer', on=["Year", "Stage"]).sort_values(by=['Year'],ascending=True)
quarter_rp.fillna(0, inplace=True)
quarter_rp.rename(columns={'Result_x':'HomeWin','Result_y':'AwayWin'},inplace=True)

In Euro 1968, There were 2 final matches beacause in the first match, Italy 1-1 Yugoslavia, and the second match was replayed then Italia won 2-0

In [ ]:

fig = px.histogram(all_matches, x="Attendance", color="Result",template='plotly_white',
                  marginal='box',opacity=0.7,nbins=35,color_discrete_sequence=['#17869E','#4F7942','#FFBF00'])
fig.update_layout(
    font_family='monospace',
    title=dict(text='Whether Attendance Affect Result',x=0.53,y=0.95,
               font=dict(color='#636363',size=20)),
    xaxis_title_text='Attendance',
    yaxis_title_text='Count',
    legend=dict(x=1,y=0.96,bordercolor='#DADADA',borderwidth=0,tracegroupgap=5),
    bargap=0.3,
)
iplot(fig)

In [ ]:
final_result['HomeTeamName'] = final_result['HomeTeamName'].apply(lambda x:x.strip('\xa0'))
final_result['AwayTeamName'] = final_result['AwayTeamName'].apply(lambda x:x.strip('\xa0'))
df = final_result[(final_result['Stage'] == 'Semi-finals')|(final_result['Stage']=='Final')]
df

In [ ]:
#List team win in semi and final
list_hw = df[df['Result']=='H']['HomeTeamName'].unique()
list_aw = df[df['Result']=='A']['AwayTeamName'].unique()
print('list_hw: ',list_hw)
print('list_aw: ',list_aw)
not_semi_final = all_matches[~((all_matches['Stage']=='Semi-finals')|(all_matches['Stage']=='Final'))] 
not_semi_final['HomeTeamName'] = not_semi_final['HomeTeamName'].apply(lambda x:x.strip('\xa0'))
not_semi_final['AwayTeamName'] = not_semi_final['AwayTeamName'].apply(lambda x:x.strip('\xa0'))

In [ ]:
list_team = {}
def extract_teamlist(team_when_hw,team_when_aw,list_team_type): #List team win and lose in semi-finals and final
    list_1 = df[df['Result']=='H'][str(team_when_hw)].unique()
    list_2 = df[df['Result']=='A'][str(team_when_aw)].unique()
    not_semi_final = all_matches[~((all_matches['Stage']=='Semi-finals')|(all_matches['Stage']=='Final'))]
    list_team[str(list_team_type)] = list(set(list_1) | set(list_2))  

not_semi_final['HomeTeamName'] = not_semi_final['HomeTeamName'].apply(lambda x:x.strip('\xa0'))
not_semi_final['AwayTeamName'] = not_semi_final['AwayTeamName'].apply(lambda x:x.strip('\xa0'))

In [ ]:
#get team win
extract_teamlist('HomeTeamName','AwayTeamName','list_teamwin')
#get team lose
extract_teamlist('AwayTeamName','HomeTeamName','list_teamlose')

In [ ]:
list_team

In [ ]:
#return the number of goals of team win and team lose in previous matches
temp_dta = pd.DataFrame()

def tw_performance(list_team_type,team_position,result_team,team_data,team_position_goals):
    dict_performance = {}
    df  = not_semi_final[not_semi_final['Result']==str(result_team)]
    for tw in list_team_type:
        if df[str(team_position)].str.contains(str(tw)).any():
            count = df[str(team_position)].str.contains(str(tw)).sum()
            dta = df[df[str(team_position)]== str(tw)]
            dict_performance[tw] = [count, dta[str(team_position_goals)].sum()] 
        else:
            continue 
        team_data = pd.DataFrame.from_dict(dict_performance, orient='index', columns=['Num_matches','Num_Goals'])
    return team_data.reset_index()

In [ ]:
r1 = tw_performance(list_team['list_teamwin'],'HomeTeamName','H',temp_dta,'HomeTeamGoals')
r2 = tw_performance(list_team['list_teamwin'],'AwayTeamName','A',temp_dta,'AwayTeamGoals')
combine_r1r2 = pd.merge(r1,r2,on=['index'],how='outer')

In [ ]:
combine_r1r2

In [ ]:
r3 = tw_performance(list_team['list_teamwin'],'HomeTeamName','D',temp_dta,'HomeTeamGoals')
r4 = tw_performance(list_team['list_teamwin'],'AwayTeamName','D',temp_dta,'AwayTeamGoals')
combine_r3r4 = pd.merge(r3,r4,on=['index'],how='outer')

In [ ]:
combine_r3r4

In [ ]:
r5 = tw_performance(list_team['list_teamlose'],'AwayTeamName','H',temp_dta,'AwayTeamGoals')
r6 = tw_performance(list_team['list_teamlose'],'HomeTeamName','A',temp_dta,'HomeTeamGoals')
combine_r5r6 = pd.merge(r5,r6,on=['index'],how='outer')
combine_r5r6

In [ ]:
r7 = tw_performance(list_team['list_teamlose'],'AwayTeamName','D',temp_dta,'AwayTeamGoals')
r8 = tw_performance(list_team['list_teamlose'],'HomeTeamName','D',temp_dta,'HomeTeamGoals')
combine_r7r8 = pd.merge(r7,r8,on=['index'],how='outer')
combine_r7r8

In [ ]:
# team win
combine_win_all = pd.merge(combine_r1r2,combine_r3r4,on=['index'],how='outer')
combine_win_all.fillna(0,inplace=True)
combine_win_all['Sum_goals'] = combine_win_all.filter(regex='^Num_Goals',axis=1).sum(axis=1)
combine_win_all['Sum_matches'] = combine_win_all.filter(regex='^Num_matches',axis=1).sum(axis=1)
combine_win_all.iloc[:,1:]=combine_win_all.iloc[:,1:].astype('int64')
combine_win_all.drop(columns=list(combine_win_all.filter(regex='^Num',axis=1).columns),axis=1,inplace=True)
combine_win_all.rename(columns={'index':'TeamName'},inplace=True)
combine_win_all['Goals_per_match'] = round(combine_win_all['Sum_goals']/combine_win_all['Sum_matches'],2)
combine_win_all.sort_values(by='Sum_goals', ascending=True,inplace=True)
#team lose
combine_lose_all = pd.merge(combine_r5r6,combine_r7r8,on=['index'],how='outer')
combine_lose_all.fillna(0,inplace=True)
combine_lose_all['Sum_goals'] = combine_lose_all.filter(regex='^Num_Goals',axis=1).sum(axis=1)
combine_lose_all['Sum_matches'] = combine_lose_all.filter(regex='^Num_matches',axis=1).sum(axis=1)
combine_lose_all.iloc[:,1:]=combine_lose_all.iloc[:,1:].astype('int64')
combine_lose_all.drop(columns=list(combine_lose_all.filter(regex='^Num',axis=1).columns),axis=1,inplace=True)
combine_lose_all.rename(columns={'index':'TeamName'},inplace=True)
combine_lose_all['Goals_per_match'] = round(combine_lose_all['Sum_goals']/combine_lose_all['Sum_matches'],2)
combine_lose_all.sort_values(by='Sum_goals', ascending=True,inplace=True)

In [ ]:
combine_lose_all

In [ ]:
from plotly.subplots import make_subplots

# Creating two subplots
fig = make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                    shared_yaxes=False, vertical_spacing=0.001)

fig.append_trace(go.Bar(
    x=combine_win_all['Sum_goals'],
    y=combine_win_all['TeamName'],
    marker=dict(
        color='rgba(50, 171, 96, 0.6)',
        line=dict(
            color='rgba(50, 171, 96, 1.0)',
            width=1),
    ),
    name='The number of goals that each team win scored',
    orientation='h',
), 1, 1)

fig.append_trace(go.Bar(
    x=combine_lose_all['Sum_goals'],
    y=combine_lose_all['TeamName'],
    marker=dict(
        color='rgba(10, 135, 30, 0.2)',
        line=dict(
            color='rgba(14, 121, 30, 1.0)',
            width=1),
    ),
    name='The number of goals that each team lose scored',
    orientation='h',
), 1, 2)

fig.update_layout(
    title='''Goals per semi-final and final stage's team win and team lose in previous stage''',
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        domain=[0, 0.9],
    ),
    yaxis2=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        linecolor='rgba(102, 102, 102, 0.8)',
        linewidth=2,
        domain=[0, 0.9],
    ),
    xaxis=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0, 0.42],
    ),
    xaxis2=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0.55, 1],
    ),
    legend=dict(x=0.029, y=1.1, font_size=10),
    margin=dict(l=100, r=20, t=70, b=70),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
)

annotations = []

# Adding labels
for ydn, yd, xd,xdn in zip(combine_lose_all['Sum_goals'], combine_win_all['Sum_goals'], combine_win_all['TeamName'],combine_lose_all['TeamName']):
    # labeling the scatter savings
    annotations.append(dict(xref='x2', yref='y2',
                            y=xdn, x=ydn + 1 ,
                            text= str(ydn),
                            font=dict(family='Arial', size=12,
                                      color='rgb(128, 0, 128)'),
                            showarrow=False))
    # labeling the bar net worth
    annotations.append(dict(xref='x1', yref='y1',
                            y=xd, x=yd + 3 ,
                            text=str(yd),
                            font=dict(family='Arial', size=12,
                                      color='rgb(35,7,43)'),
                            showarrow=False))
                           

     
# Source
annotations.append(dict(xref='paper', yref='paper',
                        x=-0.2, y=-0.109,
                        font=dict(family='Arial', size=20, color='rgb(150,150,150)'),
                        showarrow=False))

fig.update_layout(annotations=annotations)

iplot(fig)

In [ ]:
# Creating two subplots
fig = make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                    shared_yaxes=False, vertical_spacing=0.001)

fig.append_trace(go.Scatter(
    x=combine_win_all['Goals_per_match'], y=combine_win_all['TeamName'],
    mode='lines+markers',
    line_color='rgb(128, 0, 128)',
    name='The rate of goals that each team win scored per match in stages (excluding semi-final and final)',
), 1, 1)

fig.append_trace(go.Scatter(
    x=combine_lose_all['Goals_per_match'], y=combine_lose_all['TeamName'],
    mode='lines+markers',
    line_color='rgb(115,115,115)',
    name='The rate of goals that each team lose scored per match in stages (excluding semi-final and final)',
), 1, 2)

fig.update_layout(
    title="""The rate of goals that both team win and team lose scored per match""",
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        domain=[0, 0.95],
    ),
    yaxis2=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        linecolor='rgba(102, 102, 102, 0.8)',
        linewidth=2,
        domain=[0, 0.95],
    ),
    xaxis=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0, 0.42],
    ),
xaxis2=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0.6, 1],
    ),
    legend=dict(x=0.029, y=1.1, font_size=10),
    margin=dict(l=100, r=20, t=70, b=70),
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
)

annotations = []
# Adding labels
for ydn, yd, xd, xdn in zip(combine_lose_all['Goals_per_match'], combine_win_all['Goals_per_match'], combine_win_all['TeamName'],combine_lose_all['TeamName']):
    # labeling the scatter savings
    annotations.append(dict(xref='x2', yref='y2',
                            y=xdn, x=ydn - 0.1,
                            text='{:,}'.format(ydn),
                            font=dict(family='Arial', size=10,
                                      color='rgb(150, 150, 150)'),
                            showarrow=False))
    annotations.append(dict(xref='x1', yref='y1',
                            y=xd, x=yd - 0.5,
                            text='{:,}'.format(yd),
                            font=dict(family='Arial', size=10,
                                      color='rgb(128, 0, 128)'),
                            showarrow=False))

fig.update_layout(annotations=annotations)

iplot(fig)

In [ ]:
print('Descriptive statistic about team win: ')
print(combine_win_all[['Goals_per_match']].describe())
print(' ')
print('Desriptive statistic about team lose: ')
print(combine_lose_all[['Goals_per_match']].describe())

# **We begin to explore more details about top 5 team**

In [ ]:

top_5 = general_stat.sort_values(by=['Points'],ascending=False,).head()
top_5['Goal_Difference'] = top_5['Goal_Difference'].astype('int64')
top_5

In [ ]:
top_5.iloc[0,1:].values

In [ ]:
top_5.describe()

In [ ]:
list(top_5.iloc[:,1:].columns)

In [ ]:

colors = ['rgba(38, 24, 74, 0.8)', 'rgba(71, 58, 131, 0.8)',
          'rgba(122, 120, 168, 0.8)', 'rgba(164, 163, 204, 0.85)',
          'rgba(190, 192, 213, 1)','rgba(12, 24, 4, 0.8)', 'rgba(49, 5, 13, 0.8)',
          'rgba(1, 120, 168, 0.8)', 'rgba(122, 16, 204, 0.85)',
          'rgba(10, 132, 23, 1)']

top_labels = list(top_5.iloc[:,1:10].columns)
x_data = [list(top_5.iloc[i,1:10]) for i in range(5)]
y_data = [top_5.iloc[i,0] for i in range(5)]

fig = go.Figure()

for i in range(0, 9):
    for xd, yd in zip(x_data, y_data):
        fig.add_trace(go.Bar(
            x=[xd[i]], y=[yd],
            orientation='h',
            marker=dict(
                color=colors[i],
                line=dict(color='rgb(248, 248, 249)', width=1)
            )
        ))

fig.update_layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.1, 1]
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    margin=dict(l=120, r=10, t=140, b=80),
    showlegend=False,
)

annotations = []

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.14, y=yd,
                            xanchor='right',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      color='rgb(67, 67, 67)'),
                            showarrow=False, align='right'))
    # labeling the first percentage of each bar (x_axis)
    annotations.append(dict(xref='x', yref='y',
                            x=xd[0]/2, y=yd,
                            text=str(xd[0]),
                            font=dict(family='Arial', size=14,
                                      color='rgb(248, 248, 255)'),
                            showarrow=False))
    # labeling the first Likert scale (on the top)
    if yd == y_data[-1]:
        annotations.append(dict(xref='x', yref='paper',
                                x=xd[0] / 2, y=1.4,
                                text=top_labels[0],
                                align = 'center',
                                textangle = -60,
                                font=dict(family='Arial', size=14
                                ,
                                          color='rgb(67, 67, 67)'),
                                showarrow=False))
    space = xd[0]
    for i in range(1, len(xd)):
            # labeling the rest of percentages for each bar (x_axis)
            annotations.append(dict(xref='x', yref='y',
                                    x=space + (xd[i]/2), y=yd,
                                    text=str(xd[i]),
                                    font=dict(family='Arial', size=13,
                                              color='rgb(248, 248, 255)'),
                                    showarrow=False))
            # labeling the Likert scale
            if yd == y_data[-1]:
                annotations.append(dict(xref='x', yref='paper',
                                        x=space + (xd[i]/2), y=1.4,
                                        text=top_labels[i],
                                        textangle = -60,
                                        font=dict(family='Arial', size=13,
                                                  color='rgb(67, 67, 67)'),
                                        showarrow=False))
            space += xd[i]

fig.update_layout(annotations=annotations)
iplot(fig)

In [ ]:
top_5

In [ ]:
all_matches['HomeTeamName'] = all_matches['HomeTeamName'].apply(lambda x:x.replace('\xa0',""))
all_matches['AwayTeamName'] = all_matches['AwayTeamName'].apply(lambda x:x.replace('\xa0',""))

# function which get total goals of each team in each season
def goal_each_season(year,team_name):
    tempor_match = all_matches[all_matches['Year'] == year]
    if tempor_match['HomeTeamName'].str.contains(str(team_name)).any() or tempor_match['AwayTeamName'].str.contains(str(team_name)).any(): 
        tempor_match = tempor_match[(tempor_match['HomeTeamName']==str(team_name)) | (tempor_match['AwayTeamName']==str(team_name))]
        if tempor_match['HomeTeamName'].str.contains(str(team_name)).any():
            home_goals = tempor_match[tempor_match['HomeTeamName']==str(team_name)]['HomeTeamGoals'].sum()
            if tempor_match['AwayTeamName'].str.contains(str(team_name)).any(): 
                away_goals = tempor_match[tempor_match['AwayTeamName']==str(team_name)]['AwayTeamGoals'].sum()
            else:
                away_goals = 0
        else:
            home_goals = 0
            if tempor_match['AwayTeamName'].str.contains(str(team_name)).any():
                away_goals = tempor_match[tempor_match['AwayTeamName']==str(team_name)]['AwayTeamGoals'].sum()
            else:
                away_goals = 0
        total_goals_thaty = home_goals + away_goals
        return total_goals_thaty    
    else:
        return None

# function that count the number of matches
def count_match_season(year,team_name):
    tempor_match = all_matches[all_matches['Year'] == year]
    if tempor_match['HomeTeamName'].str.contains(str(team_name)).any() or tempor_match['AwayTeamName'].str.contains(str(team_name)).any(): 
        tempor_match = tempor_match[(tempor_match['HomeTeamName']==str(team_name)) | (tempor_match['AwayTeamName']==str(team_name))]
        count_match_s = len(tempor_match)
        return count_match_s
    else:
        return None

In [ ]:
list_5team = list(top_5['Team'].values)

r = {}
name = []
year = []
goal = []
# goal
for team in list_5team:
    for i in range(1960,2019,4):
        name.append(team)
        year.append(i)
        goal.append(goal_each_season(i,team))
#count
count_s = []
for team in list_5team:
    for i in range(1960,2019,4):
        count_s.append(count_match_season(i,team))
r = {'TeamName':name,'Year':year,'Goal':goal, 'Num_of_match':count_s}
top5_perform = pd .DataFrame(r, columns=['TeamName','Year','Goal','Num_of_match'])
top5_perform.dropna(inplace=True)
top5_perform[['Goal','Num_of_match']] = top5_perform[['Goal','Num_of_match']].astype('int64')    
top5_perform = top5_perform[top5_perform['Num_of_match'] != 0]

In [ ]:
fig = go.Figure()
for team in list_5team:
    fig.add_trace(go.Scatter(x=top5_perform[top5_perform['TeamName']==str(team)]['Year'], y=top5_perform[top5_perform['TeamName']==str(team)]['Goal'], name=str(team)))
fig.update_layout(template="plotly_dark",title='The Number Of Goal Scored By Each 5 Top Team In Each Season',legend=dict(y=0.5, traceorder='reversed', font_size=16))
iplot(fig)

In [ ]:
fig = go.Figure()
for team in list_5team:
    fig.add_trace(go.Scatter(x=top5_perform[top5_perform['TeamName']==str(team)]['Year'], y=top5_perform[top5_perform['TeamName']==str(team)]['Num_of_match'], name=str(team)))
fig.update_layout(template="plotly_dark",title = 'The Number Of Matches Played By Each 5 Top Team In Each Season',legend=dict(y=0.5, traceorder='reversed', font_size=16))

iplot(fig)

# **Feature Selection, Prepare Final Data For Machine Learning**

In [ ]:
fifa_ranking = pd.read_csv(r'../input/fifa-rank/fifa_ranking.csv') 
fifa_ranking = fifa_ranking[fifa_ranking['confederation'] == 'UEFA']
fifa_ranking.drop(columns=['id','country_abrv','confederation'], inplace=True)
fifa_ranking

In [ ]:
euro_team = list(set(all_matches['HomeTeamName'].unique())|set(all_matches['AwayTeamName'].unique()))
fifa_ranking = fifa_ranking[fifa_ranking['country_full'].isin(euro_team)]
fifa_ranking['rank_date'] = pd.to_datetime(fifa_ranking['rank_date'])
fifa_ranking['rank_year'] = fifa_ranking['rank_date'].dt.year

In [ ]:
fifa_ranking

- country_full — country full name
- rank — current country rank
- total_points — current total points
- previous_points — total points in last rating
- rank_change — how rank has changed since the last publication
- rank_date — date of rating calculation

In [ ]:
current_euro_team = list(set(fifa_ranking['country_full'].unique())&set(euro_team))
diff_euro_team = list(set(fifa_ranking['country_full'].unique())^set(euro_team))

In [ ]:
all_matches['Date'] = pd.to_datetime(all_matches['Date'])
matches_1996_2016 = all_matches[all_matches['Date'] > fifa_ranking['rank_date'].min()]
matches_1996_2016

In [ ]:
matches_1996_2016.drop(matches_1996_2016[matches_1996_2016['HomeTeamName'].isin(diff_euro_team)].index[0:2], inplace=True)
matches_1996_2016.drop(matches_1996_2016[matches_1996_2016['AwayTeamName'].isin(diff_euro_team)].index[0:2], inplace=True)

In [ ]:
list_team_todf = []
for year in range(1996,2019,4):
    year_ranking = fifa_ranking[fifa_ranking['rank_year'] == year]
    year_match = matches_1996_2016[matches_1996_2016['Year'] == year]
    rank_list_home = []
    point_list_home = []
    rank_list_away = []
    point_list_away = []
    for n in range(len(year_match)):
        home = []
        away = []
        home_name = year_ranking[year_ranking['country_full'] == year_match['HomeTeamName'].iloc[n]]
        away_name = year_ranking[year_ranking['country_full'] == year_match['AwayTeamName'].iloc[n]]
                # handle with HomeTeamName column
        for i in range(len(home_name)):    
            home.append(abs(year_match['Date'].iloc[n] - home_name['rank_date'].iloc[i]))#compared to the nearest day in fifa ranking
            idx = np.argmin(home)
        rank_list_home.append(home_name.iloc[idx]['rank'])
        point_list_home.append(home_name.iloc[idx]['total_points'])
                # handle with AwayTeamName column
        for i in range(len(away_name)):    
            away.append(abs(year_match['Date'].iloc[n] - away_name['rank_date'].iloc[i])) #compared to the nearest day in fifa ranking
            idx = np.argmin(away)
        rank_list_away.append(away_name.iloc[idx]['rank'])
        point_list_away.append(away_name.iloc[idx]['total_points'])
    year_match['home_team_rank'] = rank_list_home
    year_match['away_team_rank'] = rank_list_away
    year_match['home_team_point'] = point_list_home
    year_match['away_team_point'] = point_list_away
    list_team_todf.append(year_match)

In [ ]:
final_data = pd.concat(list_team_todf)
final_data.drop(columns=['Stage','SpecialWinConditions','Stadium'], inplace = True)
final_data['dif_rank'] = final_data['home_team_rank'] - final_data['away_team_rank']
final_data['dif_point'] = final_data['home_team_point'] - final_data['away_team_point']
final_data['City'] = final_data['City'].apply(lambda x:x.strip())

In [ ]:
country_match_held = {'England':['London', 'Birmingham', 'Leeds', 'Newcastle', 'Manchester','Liverpool', 'Sheffield', 'Nottingham'],'Belgium':['Liège','Charleroi','Brussels','Bruges'],'Nertherlands':['Eindhoven','Arnhem','Rotterdam','Amsterdam'],'Portugal':['Porto','Faro/Loulé','Lisbon','Leiria','Coimbra','Guimarães', 'Braga','Aveiro'],'Switzerland':['Basel', 'Geneva','Zürich','Bern'],'Austria':['Vienna','Klagenfurt','Innsbruck', 'Salzburg'],'Poland':['Warsaw','Wrocław','Gdańsk','Poznań'],'Ukraine':['Kharkiv','Lviv','Donetsk','Kiev'],'France':['Saint-Denis', 'Lens', 'Paris', 'Marseille','Décines-Charpieu', "Villeneuve-d'Ascq", 'Bordeaux', 'Toulouse','Saint-Étienne', 'Nice']}

In [ ]:
team_to_replace = ['England','Belgium', 'Nertherlands','Portugal','Switzerland','Austria','Poland','Ukraine','France']
for team in team_to_replace:
    final_data['City'] = final_data['City'].replace(country_match_held[team],team)
final_data.rename(columns={'City':'Country_held'}, inplace=True)

In [ ]:
list_status = []
for i in range(len(final_data)):
    if final_data.iloc[i,1] == final_data.iloc[i,5]:
        list_status.append('Home')
    elif final_data.iloc[i,2] == final_data.iloc[i,5]:
        list_status.append('Away')
    else:
        list_status.append('Neutral')
final_data['Status'] = list_status
final_data.drop(columns=['Country_held'], inplace=True)

# ** Create train, test set**

In [ ]:
final_data.drop(columns = ['Date','Year'],inplace=True)

# using Stratified shuffle to make sure that the rate of each result equally
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(final_data, final_data['Result']):
    strat_train_set = final_data.iloc[train_index]
    strat_test_set = final_data.iloc[test_index]

# ** Looking for Correlations**

In [ ]:
from pandas.plotting import scatter_matrix
# copy stat train data to play without harming the training set
ml_data = strat_train_set.copy()
# plot correlation some promising variables
attributes = ['Attendance','home_team_rank','away_team_rank','home_team_point','away_team_point','dif_rank','dif_point']
scatter_matrix(ml_data[attributes], figsize=(12,8))

In [ ]:
corr_matrix = ml_data[attributes].corr()
plt.subplots(figsize=(9, 9))
sns.heatmap(corr_matrix, annot=True, vmax=1, square=True, cmap="Blues")
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(1,2,1)
plt.scatter(ml_data['HomeTeamGoals']-ml_data['AwayTeamGoals'], ml_data['dif_rank'])
plt.title('Difference btw diff_rank and diff_goals')
plt.ylabel('Distance from Home rank to Away rank')
plt.xlabel('Distance from Home Goals to Away goal')


plt.subplot(1,2,2)
plt.scatter(ml_data['HomeTeamGoals']-ml_data['AwayTeamGoals'], ml_data['dif_point'])
plt.title('Difference btw diff_point in rank and diff_goals')
plt.ylabel('Distance from Home point to Away point')
plt.xlabel('Distance Home Goals to Awayy goal')

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(1,2,1)
plt.bar(ml_data['HomeTeamGoals'].value_counts().index,ml_data['HomeTeamGoals'].value_counts().values)
plt.title('The Number Of Goals Home Team Scored')
plt.xlabel('Goals')
plt.ylabel('Count')

plt.subplot(1,2,2)
plt.bar(ml_data['AwayTeamGoals'].value_counts().index, ml_data['AwayTeamGoals'].value_counts().values)
plt.title('The Number Of Goals Away Team Scored')
plt.xlabel('Goals')
plt.ylabel('Count')

In [ ]:
plt.scatter(ml_data['HomeTeamGoals']-ml_data['AwayTeamGoals'], ml_data['Attendance'])

In [ ]:
qq = ml_data.groupby('Status')['Result'].value_counts()
qq.plot(kind='barh')

# **Data Cleaning, Columns Transformation**

In [ ]:
f_data = final_data.drop(columns = ['Attendance'])
team_unique = list(set(f_data['HomeTeamName'].unique())&set(f_data['AwayTeamName'].unique()))

In [ ]:
f_data = pd.concat([f_data,matches_1996_2016[['Stage']]],axis = 1)

In [ ]:
f_data

In [ ]:
f_data['Stage'] = f_data['Stage'].replace(['Group A','Group B','Group C','Group D','Group E','Group F'],'Group stage')

In [ ]:
 data_prepared = f_data[(f_data['HomeTeamName'].isin(team_unique))&(f_data['AwayTeamName'].isin(team_unique))]

In [ ]:
X_data = data_prepared.drop(columns=['HomeTeamGoals','AwayTeamGoals','Result','home_team_rank','away_team_rank','home_team_point','away_team_point'])
y_data = data_prepared['Result'].copy()

In [ ]:
# columns transform
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [ ]:
q1 = OneHotEncoder().fit_transform(X_data[['HomeTeamName']]).toarray()
q2 = OneHotEncoder().fit_transform(X_data[['AwayTeamName']]).toarray()
q3 = StandardScaler().fit_transform(X_data[['dif_rank']])
q4 = StandardScaler().fit_transform(X_data[['dif_point']])
q5 = OneHotEncoder().fit_transform(X_data[['Status']]).toarray()
q6 = OneHotEncoder().fit_transform(X_data[['Stage']]).toarray()
q7 = LabelEncoder().fit_transform(y_data)

In [ ]:
X_data

In [ ]:
X = np.concatenate((q1,q2,q3,q4,q5,q6), axis=1)
y = q7

In [ ]:
X_y = np.concatenate((X,y.reshape(-1,1)), axis = 1)

In [ ]:
# using Stratified shuffle to make sure that the rate of each result equally
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(X_y, y):
    strat_train_set = X_y[train_index]
    strat_test_set = X_y[test_index]

In [ ]:
X_train = strat_train_set[:,:-1]
X_test = strat_test_set[:,:-1]
y_train = strat_train_set[:,-1]
y_test = strat_test_set[:,-1]

# **Train and Evaluate Model**

In [ ]:
# compare algorithms
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('KNN', KNeighborsClassifier()))
models.append(('SVM', SVC(gamma='auto')))
models.append(('CART', DecisionTreeClassifier()))
models.append(('XGB',xgb.XGBClassifier(seed = 82)))
models.append(('NB', GaussianNB()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('RAND',RandomForestClassifier()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model,X_train, y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

In [ ]:
# Compare Algorithms
plt.figure(figsize=(10,8))
plt.boxplot(results, labels=names)
plt.title('Algorithm Comparison')
plt.show()

# **Make Prediction**

In [ ]:
model = SVC(gamma='auto')
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [ ]:
model_1 = DecisionTreeClassifier()
model_1.fit(X_train, y_train)
predictions_1 = model_1.predict(X_test)

# **Evaluate Prediciton**

In [ ]:
# Evaluate predictions
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
# Evaluate predictions
print('')
print(accuracy_score(y_test, predictions_1))
print(confusion_matrix(y_test, predictions_1))
print(classification_report(y_test, predictions_1))

In [ ]:
from sklearn.model_selection import GridSearchCV
  
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

# print best parameter after tuning
print(grid.best_params_)

# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

In [ ]:
grid_predictions = grid.predict(X_test)
print(confusion_matrix(y_test,grid_predictions))
print(classification_report(y_test,grid_predictions))

In [ ]:
grid_1 = DecisionTreeClassifier(random_state=6).fit(X_train,y_train)
grid_predictions_1 = grid_1.predict(X_test)
print(confusion_matrix(y_test,grid_predictions_1))
print(classification_report(y_test,grid_predictions_1))

In [ ]:
X_data.describe()

In [ ]:
England = q1[0].reshape(1,30)
Italy = q2[14].reshape(1,30)
dif_rank_EvsI = np.array([-0.1938917437]).reshape(1,1) #convert dif_rank to Standard scale by hand
dif_point_EvsI = np.array([0.1618811981]).reshape(1,1) #convert dif_point to Standard scale by hand
status_gr = q5[0].reshape(1,3)
stage = q6[-1].reshape(1,5)

In [ ]:
final_euro_2021 = np.concatenate((England,Italy,dif_rank_EvsI,dif_point_EvsI,status_gr,stage),axis = 1)

In [ ]:
predict_champ = grid_1.predict(final_euro_2021) #use DecisionTree to predict
predict_champ

In [ ]:
 grid.predict(final_euro_2021) # use SVC to predict

# **According to model built, maybe England is predicted to become a winner in Euro 2021**

# Save model

In [ ]:
import pickle
filename_0 = 'predict_euro_model_SVC.sav'
filename_1 = 'predict_euro_model_treeclass.sav'
pickle.dump(grid, open(filename_0, 'wb'))
pickle.dump(grid_1, open(filename_1, 'wb'))

# **Predict happened matches in Euro 2021 to see performance**

In [ ]:
euro_2021_matches = pd.read_csv(r'../input/match-euro/euro2021_match.csv')

In [ ]:
dict_encode = {'England':q1[0],'Netherlands':q1[1],'Switzerland':q1[2],'Scotland':q1[3],'Spain':q1[6],'France':q1[9],
'Germany':q1[12],'Italy':q1[13],'Czech Republic':q1[14],'Russia':q1[15],'Denmark':q1[18],'Turkey':q1[19],
'Portugal':q1[20],'Croatia':q1[21],'Belgium':q1[37],'Sweden':q1[40],'Austria':q1[92],'Poland':q1[96],'Ukraine':q1[136],'Wales':q1[154],'Slovakia':q1[159],'Hungary':q1[183]}

In [ ]:
status_encode = {'H':q5[0],'Neutral':q5[1],'A':q5[3]}
stage_encode = {'Group stage':q6[0],'Round of 16':q6[184],'Quarter-finals':q6[192],'Semi-finals':q6[196]}

In [ ]:
opp = []
for i in range(44):
    result = np.concatenate((dict_encode[euro_2021_matches['HomeTeamName'][i]].reshape(1,30),dict_encode[euro_2021_matches['AwayTeamName'][i]].reshape(1,30),np.array(euro_2021_matches['StandardScaler dif_rank'][i]).reshape(1,1),np.array(euro_2021_matches['StandardScaler dif_point'][i]).reshape(1,1),status_encode[euro_2021_matches['Status'][i]].reshape(1,3),stage_encode[euro_2021_matches['Stage'][i]].reshape(1,5)),axis=1)
    opp.append(result)

In [ ]:
test_2021 = np.concatenate(opp,axis =0)
result_2021 = LabelEncoder().fit_transform(pd.read_csv(r'../input/result/result_euro.csv'))

In [ ]:
# experiment with DecisionTree
loaded_model = pickle.load(open(filename_1, 'rb'))
result = loaded_model.score(test_2021, result_2021)
print(result)
print(confusion_matrix(result_2021,loaded_model.predict(test_2021)))
print(classification_report(result_2021,loaded_model.predict(test_2021)))

In [ ]:
# experiment with SVC model
loaded_model = pickle.load(open(filename_0, 'rb'))
result = loaded_model.score(test_2021, result_2021)
print(result)
print(confusion_matrix(result_2021,loaded_model.predict(test_2021)))
print(classification_report(result_2021,loaded_model.predict(test_2021)))

We can see that DecisionTree Classifier predicts Home win, away win and even draw for both team, but Support Vector Machine only predicts Home win or Away win, get rid of draw results. So I think if result is in 90 minutes, DecisionTreeClassifier will be better and If result is considered to the end (120 minutes, penalties), SVC will be the best choice.